### <!--NAVIGATION-->
< [更多IPython资源](01.08-More-IPython-Resources.ipynb) | [目录](Index.ipynb) | [理解Python中的数据类型](02.01-Understanding-Data-Types.ipynb) >

<a href="https://colab.research.google.com/github/wangyingsm/Python-Data-Science-Handbook/blob/master/notebooks/02.00-Introduction-to-NumPy.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# Introduction to NumPy

# NumPy 介绍

> This chapter, along with chapter 3, outlines techniques for effectively loading, storing, and manipulating in-memory data in Python.
The topic is very broad: datasets can come from a wide range of sources and a wide range of formats, including be collections of documents, collections of images, collections of sound clips, collections of numerical measurements, or nearly anything else.
Despite this apparent heterogeneity, it will help us to think of all data fundamentally as arrays of numbers.

這個主題非常廣泛：數據集可能來自非常不同的來源和非常不同的格式，包括文檔的集合，圖像的集合，聲音片段的集合，數值測量的集合，甚至其他任何東西的集合。儘管數據集有著超出想像的異質性，我們還是可以將所有的數據抽象成為數值組成的數組。

> For example, images–particularly digital images–can be thought of as simply two-dimensional arrays of numbers representing pixel brightness across the area.
Sound clips can be thought of as one-dimensional arrays of intensity versus time.
Text can be converted in various ways into numerical representations, perhaps binary digits representing the frequency of certain words or pairs of words.
No matter what the data are, the first step in making it analyzable will be to transform them into arrays of numbers.
(We will discuss some specific examples of this process later in [Feature Engineering](05.04-Feature-Engineering.ipynb))

例如圖像，這裡我們特指數字圖像，可以被認為是簡單的二維數組，包含著代表這區域內每個像素亮度的數值。聲音片段可以被認為是一維的數組，包含著時間範圍內聲音強度的數值。文本可以使用各種方法轉換成為數值方式表示，比方說使用二進制數字表示某個單詞或短語的出現頻率。無論數據是哪種類型，我們對它們進行處理的時候，第一步總是設計將它們轉換為數值。 （參見[特徵工程](05.04-Feature-Engineering.ipynb)）

> For this reason, efficient storage and manipulation of numerical arrays is absolutely fundamental to the process of doing data science.
We'll now take a look at the specialized tools that Python has for handling such numerical arrays: the NumPy package, and the Pandas package (discussed in Chapter 3).This chapter will cover NumPy in detail. NumPy (short for *Numerical Python*) provides an efficient interface to store and operate on dense data buffers.
In some ways, NumPy arrays are like Python's built-in ``list`` type, but NumPy arrays provide much more efficient storage and data operations as the arrays grow larger in size.
NumPy arrays form the core of nearly the entire ecosystem of data science tools in Python, so time spent learning to use effectively will be valuable no matter what aspect of data science interests you.For the pieces of the package discussed here, I'd recommend NumPy version 1.8 or later.
By convention, you'll find that most people in the SciPy/PyData world will import NumPy using ``np`` as an alias:

因此，有效的存儲和處理數值數組對於數據科學來說是最根本的能力。我們接下來會討論Python中具備這樣強大功能的特殊工具：NumPy和Pandas (Numerical Python)，它提供了強大的接口供我們存儲和操作非稀疏數據集合。在某些情況下，NumPy的數組表現得就像Python內建的`列表`，但是NumPy數組在存儲和操作大量數據集合的時候提供了有效得多的功能和性能。 NumPy數組是Python的數據科學領域工具鏈的核心，很多其他的工具都是在它的基礎上構建的，因此無論你感興趣的是數據科學的哪個領域，NumPy都值得你花時間進行鑽研。

## Reminder about Built In Documentation

## A Python List Is More Than Just a List

> Let's consider now what happens when we use a Python data structure that holds many Python objects.
The standard mutable multi-element container in Python is the list.We can create a list of integers as follows: 
> As you read through this chapter, don't forget that IPython gives you the ability to quickly explore the contents of a package (by using the tab-completion feature), as well as the documentation of various functions (using the ``?`` character – Refer back to [Help and Documentation in IPython](01.01-Help-And-Documentation.ipynb)).

現在我們繼續考慮當我們使用Python的數據結構來存儲許多這樣的Python對象時的情況。 Python中標準的可變多元素的容器集合就是列表。我們按如下的方式創建一個整數的列表又或者，類似的，字符串的列表：在你閱讀本章的過程中，請不要忘記了IPython提供的內建幫助工具?以及使用製表符自動補全的功能。

In [1]:
import numpy as np
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [2]:
L2 = [str(c) for c in L] # 列表解析
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [3]:
# 因為Python是動態類型，我們甚至可以創建不同類型元素的列表：
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

> But this flexibility comes at a cost: to allow these flexible types, each item in the list must contain its own type info, reference count, and other information–that is, each item is a complete Python object.
In the special case that all variables are of the same type, much of this information is redundant: it can be much more efficient to store data in a fixed-type array.
The difference between a dynamic-type list and a fixed-type (NumPy-style) array is illustrated in the following figure:

這種靈活性是要付出代價的：要讓列表能夠容納不同的類型，每個列表中的元素都必須帶有自己的類型信息、引用計數器和其他的信息，一句話，裡面的每個元素都是一個完整的Python的對象。如果在所有的元素都是同一種類型的情況下，這裡面絕大部分的信息都是冗餘的：如果我們能將數據存儲在一個固定類型的數組中，顯然會更加高效。下圖展示了動態類型的列表和固定類型的數組（NumPy實現的）的區別：

![Array Memory Layout](https://github.com/wangyingsm/Python-Data-Science-Handbook/raw/e044d370f852cd3639bbe45ebc2eb3e6f11c1e62/notebooks/figures/array_vs_list.png)

> At the implementation level, the array essentially contains a single pointer to one contiguous block of data.
The Python list, on the other hand, contains a pointer to a block of pointers, each of which in turn points to a full Python object like the Python integer we saw earlier.
Again, the advantage of the list is flexibility: because each list element is a full structure containing both data and type information, the list can be filled with data of any desired type.
Fixed-type NumPy-style arrays lack this flexibility, but are much more efficient for storing and manipulating data.

從底層實現上看，數組僅僅包含一個指針指向一塊連續的內存空間。而Python列表，含有一個指針指向一塊連續的指針內存空間，裡面的每個指針再指向內存中每個獨立的Python對象，如我們前面看到的整數。列表的優勢在於靈活：因為每個元素都是完整的Python的類型對象結構，包含了數據和類型信息，因此列表可以存儲任何類型的數據。 NumPy使用的固定類型的數組缺少這種靈活性，但是對於存儲和操作數據會高效許多。

## Creating Arrays from Python Lists

> First, we can use ``np.array`` to create arrays from Python lists: Remember that unlike Python lists, NumPy is constrained to arrays that all contain the same type.If types do not match, NumPy will upcast if possible (here, integers are up-cast to floating point):

首先，我們可以使用`np.array`來將一個Python列表變成一個數組，記住和Python列表不同，NumPy數組只能含有同一種類型的數據。如果類型不一樣，NumPy會嘗試向上擴展類型（下面例子中會將整數向上擴展為浮點數）：

In [4]:
np.array([1, 4, 2, 5, 3])  # 整數

array([1, 4, 2, 5, 3])

In [5]:
np.array([3.14, 4, 2, 3])  # 擴展

array([3.14, 4.  , 2.  , 3.  ])

In [6]:
np.array([1, 2, 3, 4], dtype='float32') # 如果需要明確定義可以使用dtype

array([1., 2., 3., 4.], dtype=float32)

### here's one way of initializing a multidimensional array using a list of lists:
### 下面例子是一個使用列表的列表來創建二維數組的方法：

In [7]:
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

In [8]:
a = np.array([0, 1, 2, 3])
a

array([0, 1, 2, 3])

### Creating 1-D array

In [9]:
import numpy as np
ar=np.array([1,4,6,3,7])
ar

array([1, 4, 6, 3, 7])

In [10]:
ar=np.array([1,4,6,3,7])
ar[2]

6

In [11]:
ar1=np.array((2,3,4,5))
ar1

array([2, 3, 4, 5])

### Creating 2-D array

In [12]:
arr=np.array([[1,2,4], [5,6,8]])
arr

array([[1, 2, 4],
       [5, 6, 8]])

In [13]:
arr=np.array([[1,2,4], [5,6,8]])
arr[0,1]

2

### Creating 3-D array

In [14]:
arr=np.array([[[1,2,3], [5,6,7]], [[1,2,3], [5,6,7]]])
arr

array([[[1, 2, 3],
        [5, 6, 7]],

       [[1, 2, 3],
        [5, 6, 7]]])

## Creating Arrays from Scratch

> Especially for larger arrays, it is more efficient to create arrays from scratch using routines built into NumPy.
Here are several examples:

使用NumPy的方法從頭創建數組會更加高效，特別對於大型數組來說。

In [15]:
# zeros將數組元素都填充為0，10是數組長度
np.zeros((3,4))
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
# ones將數組元素都填充為1，(3, 5)是數組的維度說明，表明數組是二維的3行5列
np.ones((2,3))
np.ones((3, 5), dtype=float)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [17]:
# full將數組元素都填充為參數值3.14，(3, 5)是數組的維度說明，表明數組是二維的3行5列
np.full((2,3), 76)
np.full((3, 5), 3.14)

array([[3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14]])

In [18]:
# arange類似range，創建一段序列值 起始值是0（包含），結束值是20（不包含），步長為2
np.arange(4)
np.arange(0, 20, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [19]:
# linspace創建一段序列值，其中元素按照區域進行線性（平均）劃分，起始值是0（包含），結束值是1（包含），共5個元素
np.linspace(0, 1, 5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

### Random numbers in array

In [20]:
# Random numbers from [0,1]
np.random.random((3,4))

array([[0.25030686, 0.60760759, 0.7821651 , 0.7028614 ],
       [0.6064729 , 0.37374075, 0.78488692, 0.33846433],
       [0.97821702, 0.25562375, 0.1134704 , 0.99231203]])

In [21]:
# random number from interval [6,80)
#To obtain a number in the interval [a,b), you can multiply above with (b-a) and then add a.
74*np.random.random((3,4))+6

array([[62.11665235, 58.46122024, 30.86478306, 42.0375365 ],
       [10.75394766, 38.16058828, 63.11182042, 46.79218183],
       [44.68999071, 33.26971181, 15.45399907, 26.68932771]])

In [22]:
#random integers from interval [6,80)
np.random.randint(high=80,low=6,size=(3,4))

array([[54, 63, 65, 69],
       [43, 18, 55, 19],
       [33, 67, 12, 46]])

In [23]:
#works only for 1D array input
a=np.array([52,-3, 48, 69, 32, 100, -12, 8])
np.random.choice(a,5,replace=False)

array([-12,  52,  69, 100,  48])

In [24]:
#to repeat the same random choices everytime, use seed
np.random.seed(2)
np.random.choice(a,5,replace=False)

array([ 32,  -3, -12,  48,  69])

In [25]:
#random sampling with replacement
np.random.choice(np.arange(8,58),(4,3),replace=True)

array([[42, 57, 39],
       [19, 29, 55],
       [39, 34, 28],
       [45, 47, 11]])

In [26]:
a=np.array([52,-3, 48, 69, 32, 100, -12, 8])
np.random.choice(a,5,replace=True)

array([-12,  32,  48,  69,  69])

In [27]:
# random number from interval [6,80)
np.random.random((3,4))

array([[0.78533515, 0.85397529, 0.49423684, 0.84656149],
       [0.07964548, 0.50524609, 0.0652865 , 0.42812233],
       [0.09653092, 0.12715997, 0.59674531, 0.226012  ]])

In [28]:
# random.random隨機分佈創建數組，隨機值範圍為[0, 1)，(3, 3)是維度說明，二維數組3行3列
np.random.random((3, 3))

array([[0.10694568, 0.22030621, 0.34982629],
       [0.46778748, 0.20174323, 0.64040673],
       [0.48306984, 0.50523672, 0.38689265]])

In [29]:
# random.normal正態分佈創建數組，均值0，標準差1，(3, 3)是維度說明，二維數組3行3列
np.random.normal(0, 1, (3, 3))

array([[ 3.70444537e-01,  1.35963386e+00,  5.01857207e-01],
       [-8.44213704e-01,  9.76147160e-06,  5.42352572e-01],
       [-3.13508197e-01,  7.71011738e-01, -1.86809065e+00]])

In [30]:
# random.randint隨機整數創建數組，隨機數範圍[0, 10)
np.random.randint(0, 10, (3, 3))

array([[8, 2, 9],
       [6, 5, 6],
       [6, 6, 3]])

In [31]:
# 3x3的單位矩陣數組
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [32]:
# empty創建一個未初始化的數組，數組元素的值保持為原有的內存空間值
np.empty(3)

array([1., 1., 1.])

In [33]:
# Create a 10x10 array with random values with seed value 7 and find the minimum and maximum values for each row.
np.random.seed(7)
x = np.random.random((10,10))
min_per_row=x.min(axis=1)
max_per_row=x.max(axis=1)

# The Basics of NumPy Arrays

> Data manipulation in Python is nearly synonymous with NumPy array manipulation: even newer tools like Pandas ([Chapter 3](03.00-Introduction-to-Pandas.ipynb)) are built around the NumPy array.
This section will present several examples of using NumPy array manipulation to access data and subarrays, and to split, reshape, and join the arrays.
While the types of operations shown here may seem a bit dry and pedantic, they comprise the building blocks of many other examples used throughout the book.
Get to know them well!

Python中的數據操作基本就是NumPy數組操作的同義詞：一些新的工具像Pandas（[第三章](03.00-Introduction-to-Pandas.ipynb)）都是依賴於NumPy數組建立起來的。本節會展示使用NumPy數組操作和訪問數據以及子數組的一些例子，包括切分、變形和組合。儘管這裡展示的操作有些枯燥和學術化，但是它們是組成本書後面使用的例子的基礎。你應該更好的掌握它們。

> - *Attributes*: Determining the size, shape, memory consumption, and data types of arrays
> - *Indexing*: Getting and setting the value of individual array elements
> - *Slicing*: Getting and setting smaller subarrays within a larger array
> - *Reshaping*: Changing the shape of a given array
> - *Joining and splitting*: Combining multiple arrays into one, and splitting one array into many

## NumPy Array Attributes

> First let's discuss some useful array attributes.
We'll start by defining three random arrays, a one-dimensional, two-dimensional, and three-dimensional array.
We'll use NumPy's random number generator, which we will *seed* with a set value in order to ensure that the same random arrays are generated each time this code is run:

首先我們來討論一些數組有用的屬性。我們從定義三個數組開始，一個一維的，一個二維的和一個三維的數組。我們採用NumPy的隨機數產生器來創建數組，產生之前我們會給定一個隨機種子，這樣來保證每次代碼運行的時候都能得到相同的數組：

In [34]:
import numpy as np
np.random.seed(0)  # 設定隨機種子，保證實驗的可重現

x1 = np.random.randint(10, size=6)
x2 = np.random.randint(10, size=(3, 4))
x3 = np.random.randint(10, size=(3, 4, 5))

In [35]:
print("x3 ndim: ", x3.ndim)  #數組的維度
print("x3 shape:", x3.shape) #數組的形狀
print("x3 size: ", x3.size)  #數組的長度
print("x3 itemsize:", x3.itemsize, "bytes")
print("x3 nbytes:", x3.nbytes, "bytes")

x3 ndim:  3
x3 shape: (3, 4, 5)
x3 size:  60
x3 itemsize: 8 bytes
x3 nbytes: 480 bytes


## Array Slicing: Accessing Subarrays
## 數組切片：獲取子數組

> Just as we can use square brackets to access individual array elements, we can also use them to access subarrays with the *slice* notation, marked by the colon (``:``) character.
The NumPy slicing syntax follows that of the standard Python list; to access a slice of an array ``x``, use this: If any of these are unspecified, they default to the values ``start=0``, ``stop=``*``size of dimension``*, ``step=1``.

正如我們可以使用中括號獲取單個元素值，我們也可以使用中括號的*切片*語法獲取子數組，切片的語法遵從標準Python列表的切片語法格式；對於一個數組`x`進行切片：如果三個參數沒有設置值的話，默認值分別是`start=0`，`stop=`*`維度的長度`*，`step=1`。
```python
x[start:stop:step]
```

### One-dimensional subarrays 一維子數組

In [36]:
x = np.arange(10)
x[:5]  # 前五個元素

array([0, 1, 2, 3, 4])

In [37]:
x[5:]  

array([5, 6, 7, 8, 9])

In [38]:
x[4:7]  

array([4, 5, 6])

In [39]:
x[::2]  # 每隔一個取元素

array([0, 2, 4, 6, 8])

In [40]:
x[1::2] # 每隔一個取元素，開始序號為1

array([1, 3, 5, 7, 9])

> A potentially confusing case is when the ``step`` value is negative.
In this case, the defaults for ``start`` and ``stop`` are swapped.
This becomes a convenient way to reverse an array:

當step為負值時，將會在數組裡反向的取元素，這是將數組反向排序最簡單的方法：

```python
s = 'hello world'
# 下面就會輸出'dlrow olleh'
print(s[::-1])
```

In [41]:
x[::-1]  # 反序數組

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

In [42]:
x[5::-2]  # 從序號5開始向前取元素，每隔一個取一個元素

array([5, 3, 1])

### Multi-dimensional subarrays 多維子數組

> Multi-dimensional slices work in the same way, with multiple slices separated by commas.
For example:

多維數組的切片也一樣，只是在中括號中使用逗號分隔多個切片聲明。例如：

In [43]:
x2

array([[3, 5, 2, 4],
       [7, 6, 8, 8],
       [1, 6, 7, 7]])

In [44]:
x2[:2, :3]  # 行的維度取前兩個，列的維度取前三個，形狀變為(2, 3)

array([[3, 5, 2],
       [7, 6, 8]])

In [45]:
x2[:3, ::2] # 行的維度取前三個（全部），列的維度每個一個取一列，形狀變為(3, 2)

array([[3, 2],
       [7, 8],
       [1, 7]])

In [46]:
x2[::-1, ::-1] # 行和列都反轉，形狀保持(3, 4)

array([[7, 7, 6, 1],
       [8, 8, 6, 7],
       [4, 2, 5, 3]])

#### Accessing array rows and columns 獲取數組的行和列

> One commonly needed routine is accessing of single rows or columns of an array.
This can be done by combining indexing and slicing, using an empty slice marked by a single colon (``:``):

還有一種常見的需要是獲取數組的單行或者單列。這可以通過組合索引和切片兩個操作做到，使用一個不帶參數的冒號`:`可以表示取該維度的所有元素：

In [47]:
print(x2[:, 0])  # x2的第一列

[3 7 1]


In [48]:
print(x2[0, :])  # x2的第一行

[3 5 2 4]


### Subarrays as no-copy views 子數組是非副本的視圖

> One important–and extremely useful–thing to know about array slices is that they return *views* rather than *copies* of the array data.
This is one area in which NumPy array slicing differs from Python list slicing: in lists, slices will be copies.
Consider our two-dimensional array from before:

一個非常重要和有用的概念你需要知道的就是數組的切片返回的實際上是子數組的*視圖*而不是它們的副本。這是NumPy數組的切片和Python列表的切片的主要區別，列表的切片返回的是副本。用上面的二維做例子：

In [49]:
print(x2)

[[3 5 2 4]
 [7 6 8 8]
 [1 6 7 7]]


> Let's extract a $2 \times 2$ subarray from this:> Now if we modify this subarray, we'll see that the original array is changed! Observe:

讓我們從中取一個$2 \times 2$的子數組：如果我們修改這個子數組，我們看到原來的數組也會隨之更改：

In [50]:
x2_sub = x2[:2, :2]
print(x2_sub)

[[3 5]
 [7 6]]


In [51]:
x2_sub[0, 0] = 99
print(x2_sub)

[[99  5]
 [ 7  6]]


In [52]:
print(x2)

[[99  5  2  4]
 [ 7  6  8  8]
 [ 1  6  7  7]]


### Creating copies of arrays 創建數組的副本

> Despite the nice features of array views, it is sometimes useful to instead explicitly copy the data within an array or a subarray. This can be most easily done with the ``copy()`` method:

儘管使用視圖有上述的優點，有時候我們還是需要從數組中復制一份子數組出來。這可以使用`copy`方法簡單的辦到：

In [53]:
x2_sub_copy = x2[:2, :2].copy()
print(x2_sub_copy)

[[99  5]
 [ 7  6]]


In [54]:
x2_sub_copy[0, 0] = 42
print(x2_sub_copy)

[[42  5]
 [ 7  6]]


In [55]:
print(x2)

[[99  5  2  4]
 [ 7  6  8  8]
 [ 1  6  7  7]]


## Reshaping of Arrays

## 改變數組的形狀

> Another useful type of operation is reshaping of arrays.
The most flexible way of doing this is with the ``reshape`` method.
For example, if you want to put the numbers 1 through 9 in a $3 \times 3$ grid, you can do the following:

另一個數組的常用操作是改變形狀。最方便的方式是使用`reshape`方法實現。例如，如果你希望將1~9的數放入一個$3 \times 3$的數組裡面，你可以這樣做：

In [56]:
grid = np.arange(1, 10).reshape((3, 3))
print(grid)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


> Note that for this to work, the size of the initial array must match the size of the reshaped array. 
Where possible, the ``reshape`` method will use a no-copy view of the initial array, but with non-contiguous memory buffers this is not always the case.

注意，改變形狀要能成功，原始數組和新的形狀的數組的總長度`size`必須一樣。當可能的情況下，`reshape`會盡量使用原始數組的視圖，但是如果原始數組的數據存儲在不連續的內存區，就會進行複制。

> Another common reshaping pattern is the conversion of a one-dimensional array into a two-dimensional row or column matrix.
This can be done with the ``reshape`` method, or more easily done by making use of the ``newaxis`` keyword within a slice operation:> We will see this type of transformation often throughout the remainder of the book.

另外一個常用的改變形狀的操作就是將一個一維數組變成二維數組中的一行或者一列。這也可以使用`reshape`方法實現，或者更簡單的方式是使用切片語法中的`newaxis`屬性增加一個維度：我們會在本書後續的內容經常看到這樣的變換。

In [57]:
x = np.array([1, 2, 3])

# 使用reshape變為 (1, 3)
x.reshape((1, 3))

array([[1, 2, 3]])

In [58]:
# 使用newaxis，增加行維度，形狀也是 (1, 3)
x[np.newaxis, :]

array([[1, 2, 3]])

In [59]:
# 使用reshape變為 (3, 1)
x.reshape((3, 1))

array([[1],
       [2],
       [3]])

In [60]:
# 使用newaxis增加列維度，形狀也是 (3, 1)
x[:, np.newaxis]

array([[1],
       [2],
       [3]])

### Reshaping data
The function we use to reshape data in NumPy is np.reshape. It takes in an array and a new shape as required arguments. The new shape must exactly contain all the elements from the input array. For example, we could reshape an array with 12 elements to (4, 3), but we can't reshape it to (4, 4).

We are allowed to use the special value of -1 in at most one dimension of the new shape. The dimension with -1 will take on the value necessary to allow the new shape to contain all the elements of the array.

The code below shows example usages of np.reshape.

In [61]:
arr = np.arange(8)

reshaped_arr = np.reshape(arr, (2, 4))
print(repr(reshaped_arr))
print('New shape: {}'.format(reshaped_arr.shape))

reshaped_arr = np.reshape(arr, (-1, 2, 2))
print(repr(reshaped_arr))
print('New shape: {}'.format(reshaped_arr.shape))

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])
New shape: (2, 4)
array([[[0, 1],
        [2, 3]],

       [[4, 5],
        [6, 7]]])
New shape: (2, 2, 2)


While the np.reshape function can perform any reshaping utilities we need, NumPy provides an inherent function for flattening an array. Flattening an array reshapes it into a 1D array. Since we need to flatten data quite often, it is a useful function.

In [62]:
arr = np.arange(8)
arr = np.reshape(arr, (2, 4))
flattened = arr.flatten()
print(repr(arr))
print('arr shape: {}'.format(arr.shape))
print(repr(flattened))
print('flattened shape: {}'.format(flattened.shape))

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])
arr shape: (2, 4)
array([0, 1, 2, 3, 4, 5, 6, 7])
flattened shape: (8,)


## Array Concatenation and Splitting

## 數組的連接和切分

> All of the preceding routines worked on single arrays. It's also possible to combine multiple arrays into one, and to conversely split a single array into multiple arrays. We'll take a look at those operations here.

前面的方法都是在單個數組上進行操作。我們也可以將多個數組組成一個，或者反過來將一個數組切分成多個。下面我們來看看這些操作。

### Concatenation of arrays 連接數組

> Concatenation, or joining of two arrays in NumPy, is primarily accomplished using the routines ``np.concatenate``, ``np.vstack``, and ``np.hstack``.
``np.concatenate`` takes a tuple or list of arrays as its first argument, as we can see here:

在NumPy中連接或者組合多個數組，有三個不同的方法`np.concatenate`，`np.vstack`和`np.hstack`。 `np.concatenate`接受一個數組的元組或列表作為第一個參數，如下：

In [63]:
arr1=np.array([2,4,6,8,2])
arr2=np.array([3,6,1,5,2])

arr1+arr2
arr1-arr2
arr1*arr2

array([ 6, 24,  6, 40,  4])

In [64]:
x = np.array([1, 2, 3])
y = np.array([3, 2, 1])
np.concatenate([x, y])

array([1, 2, 3, 3, 2, 1])

In [65]:
# You can also concatenate more than two arrays at once:
z = [99, 99, 99]
print(np.concatenate([x, y, z]))

[ 1  2  3  3  2  1 99 99 99]


In [66]:
# It can also be used for two-dimensional arrays:
grid = np.array([[1, 2, 3],
                 [4, 5, 6]])

In [67]:
# 沿著第一個維度進行連接，即按照行連接，axis=0
np.concatenate([grid, grid]) 

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6]])

In [68]:
# 沿著第二個維度進行連接，即按照列連接，axis=1
np.concatenate([grid, grid], axis=1)

array([[1, 2, 3, 1, 2, 3],
       [4, 5, 6, 4, 5, 6]])

In [69]:
one = np.array([[0, 1, 2],
               [3, 4, 5],
               [6, 7, 8]])

two = np.array([[10, 11, 12],
               [13, 14, 15],
               [16, 17, 18]])

#joining 2D arrays vertically
np.concatenate((one,two))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [10, 11, 12],
       [13, 14, 15],
       [16, 17, 18]])

> Working with arrays of mixed dimensions, it can be use the ``np.vstack`` (vertical stack) and ``np.hstack`` (horizontal stack) functions:

進行連接的數組如果具有不同的維度，使用`np.vstack`（垂直堆疊）和`np.hstack`（水平堆疊）會更加清晰：

In [70]:
one = np.arange(2,10,2)
two = np.arange(1,5)
print(one)
print()
print(two)

[2 4 6 8]

[1 2 3 4]


In [71]:
#joining 1D arrays vertically to form 2D array
np.vstack((one,two))

array([[2, 4, 6, 8],
       [1, 2, 3, 4]])

In [72]:
#joining 1D arrays horizontally, but it will output 1D array
np.hstack((one,two))

array([2, 4, 6, 8, 1, 2, 3, 4])

In [73]:
#joining 1D arrays horizontally and output 2D array
np.column_stack((one,two))

array([[2, 1],
       [4, 2],
       [6, 3],
       [8, 4]])

In [74]:
x = np.array([1, 2, 3])
grid = np.array([[9, 8, 7],
                 [6, 5, 4]])

# 沿著垂直堆疊
np.vstack([x, grid])

array([[1, 2, 3],
       [9, 8, 7],
       [6, 5, 4]])

In [75]:
# 沿著水平堆疊
y = np.array([[99],[99]])
np.hstack([grid, y])

array([[ 9,  8,  7, 99],
       [ 6,  5,  4, 99]])

In [76]:
#沿著第三個維度（深度）進行堆疊
np.dstack  

<function numpy.dstack(tup)>

In [77]:
one = np.array([[3,5],[10, 4]])
two = one + np.array([5, 2])
print(two)
np.vstack((one,two))

[[ 8  7]
 [15  6]]


array([[ 3,  5],
       [10,  4],
       [ 8,  7],
       [15,  6]])

In [78]:
one = np.array([[3,5],[10, 4]])
two = one + np.array([5, 2])
print(two)
np.hstack((one,two))

[[ 8  7]
 [15  6]]


array([[ 3,  5,  8,  7],
       [10,  4, 15,  6]])

In [79]:
x = np.zeros((5,5))
y= np.arange(5)
original=x+y
print(original)

# Create zeros col and row to append.
zeros_row=np.zeros(7,dtype=int).reshape(1,7)
zeros_col=np.zeros(5,dtype=int).reshape(5,1)

# append using vstack and hstack
padded=np.hstack((zeros_col,original,zeros_col))
padded=np.vstack((zeros_row,padded,zeros_row))
print(padded)

# Numpy also provide pad function 
print("Alternate Solution")
alternate = np.pad(original, pad_width=1, mode='constant', constant_values=0)
print(alternate)

[[0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 2. 3. 4. 0.]
 [0. 0. 1. 2. 3. 4. 0.]
 [0. 0. 1. 2. 3. 4. 0.]
 [0. 0. 1. 2. 3. 4. 0.]
 [0. 0. 1. 2. 3. 4. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
Alternate Solution
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 2. 3. 4. 0.]
 [0. 0. 1. 2. 3. 4. 0.]
 [0. 0. 1. 2. 3. 4. 0.]
 [0. 0. 1. 2. 3. 4. 0.]
 [0. 0. 1. 2. 3. 4. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


### Splitting of arrays 切分數組

> The opposite of concatenation is splitting, which is implemented by the functions ``np.split``, ``np.hsplit``, and ``np.vsplit``.  For each of these, we can pass a list of indices giving the split points:

連接的反操作是切分，主要的方法包括`np.split`，`np.hsplit`和`np.vsplit`。我們可以傳遞一個列表參數表示切分的點：

In [80]:
x = [1, 2, 3, 99, 99, 3, 2, 1]
x1, x2, x3 = np.split(x, [3, 5]) # 在序號3和序號5處進行切分，返回三個數組
print(x1, x2, x3)

[1 2 3] [99 99] [3 2 1]


> Notice that *N* split-points, leads to *N + 1* subarrays.
The related functions ``np.hsplit`` and ``np.vsplit`` are similar:

你應該已經發現N個切分點會返回N+1個子數組。相應的`np.hsplit`和`np.vsplit`也是相似的：

In [81]:
grid = np.arange(16).reshape((4, 4))
grid

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [82]:
upper, lower = np.vsplit(grid, [2]) # 沿垂直方向切分，切分点行序号为2
print(upper)
print(lower)

[[0 1 2 3]
 [4 5 6 7]]
[[ 8  9 10 11]
 [12 13 14 15]]


In [83]:
left, right = np.hsplit(grid, [2]) # 沿水平方向切分数组，切分点列序号为2
print(left)
print(right)

[[ 0  1]
 [ 4  5]
 [ 8  9]
 [12 13]]
[[ 2  3]
 [ 6  7]
 [10 11]
 [14 15]]


# Computation on NumPy Arrays: Universal Functions

# NumPy數組運算：通用函數

## The Slowness of Loops 循環，慢的實現

> The relative sluggishness of Python generally manifests itself in situations where many small operations are being repeated – for instance looping over arrays to operate on each element.
For example, imagine we have an array of values and we'd like to compute the reciprocal of each.
A straightforward approach might look like this: This implementation probably feels fairly natural to someone from, say, a C or Java background. But if we measure the execution time of this code for a large input, we see that this operation is very slow, perhaps surprisingly so! We'll benchmark this with IPython's ``%timeit`` magic (discussed in [Profiling and Timing Code](01.07-Timing-and-Profiling.ipynb)):

上面的代碼實現對於很多具有C或者Java語言背景的讀者來說是非常自然的。但是如果我們在一個很大的數據集上測量上面代碼的執行時間，我們會發現這個操作很慢，甚至慢的讓你吃驚。下面使用`%timeit`魔術指令（參見[性能測算和計時](01.07-Timing-and-Profiling.ipynb)）對一個大數據集進行測時：Python 當重複進行很多細微操作時表現相對低效，比方說一個數組中的每個元素進行循環操作。例如我們有一個數組，現在我們需要計算每個元素的倒數。

In [84]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

In [85]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

1.4 s ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


> It takes several seconds to compute these million operations and to store the result!
When even cell phones have processing speeds measured in Giga-FLOPS (i.e., billions of numerical operations per second), this seems almost absurdly slow.
It turns out that the bottleneck here is not the operations themselves, but the type-checking and function dispatches that CPython must do at each cycle of the loop.
Each time the reciprocal is computed, Python first examines the object's type and does a dynamic lookup of the correct function to use for that type.
If we were working in compiled code instead, this type specification would be known before the code executes and the result could be computed much more efficiently.

這個操作對於百萬級的數據集耗時需要幾秒。當現在手機的每秒浮點數運算次數都已經已經達到10億級別，這實在是不可思議的慢了。通過分析發現瓶頸並不是代碼本身，而是每次循環時CPython必須執行的類型檢查和函數匹配。每次計算倒數時，Python首先需要檢查對象的類型，然後尋找一個最合適的函數對這種類型進行計算。如果我們使用編譯型的語言實現上面的代碼，每次計算的時候，類型和應該執行的函數都已經確定，因此執行的時間肯定短很多。

## Introducing UFuncs UFuncs
## 介紹

> For many types of operations, NumPy provides a convenient interface into just this kind of statically typed, compiled routine. This is known as a *vectorized* operation.
This can be accomplished by simply performing an operation on the array, which will then be applied to each element.
This vectorized approach is designed to push the loop into the compiled layer that underlies NumPy, leading to much faster execution. Looking at the execution time for our big array, we see that it completes orders of magnitude faster than the Python loop:

對於許多操作，NumPy都為這種靜態類型提供了編譯好的函數。被稱為*向量化*的操作。向量化操作可以簡單應用在數組上，實際上會應用在每一個元素上。實現原理就是將循環的部分放進NumPy編譯後的那個層次，從而提高性能。下面使用ufuncs來測算執行時間，我們可以看到執行時間相差了好幾個數量級：

In [86]:
print(compute_reciprocals(values))
print(1.0 / values)

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


In [87]:
%timeit (1.0 / big_array)

591 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


> Vectorized operations in NumPy are implemented via *ufuncs*, whose main purpose is to quickly execute repeated operations on values in NumPy arrays.
Ufuncs are extremely flexible – before we saw an operation between a scalar and an array, but we can also operate between two arrays:

NumPy中的向量化操作是通過*ufuncs*實現的，其主要目的就是在NumPy數組中快速執行重複的元素操作。 Ufuncs是極端靈活的，我們上面看到是標量和數組間的操作，但是我們也可以將它們用在兩個數組之間：

In [88]:
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

In [89]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

### Array arithmetic 數組運算

> NumPy's ufuncs feel very natural to use because they make use of Python's native arithmetic operators.
The standard addition, subtraction, multiplication, and division can all be used:

NumPy的ufuncs用起來非常的自然和人性化，因為它們採用了Python本身的算術運算符號 - 標準的加法、剪髮、乘法和除法實現：

In [90]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # 整除

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [0.  0.5 1.  1.5]
x // 2 = [0 0 1 1]


> There is also a unary ufunc for negation, and a ``**`` operator for exponentiation, and a ``%`` operator for modulus:

下面是一元的取反，`**`求幂和`%`取模：看到的這些算術運算操作，都是NumPy中相應函數的簡化寫法；例如+號實際上是add函數的封裝

In [91]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


In [92]:
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

In [93]:
np.add(x, 2)

array([2, 3, 4, 5])

In [94]:
import numpy as np
arr=np.array([1,3,5,2,6])
np.cos(arr)
np.median(arr)
np.mean(arr)
np.std(arr)

1.8547236990991407

> he following table lists the arithmetic operators implemented in NumPy: Additionally there are Boolean/bitwise operators; we will explore these in [Comparisons, Masks, and Boolean Logic](02.06-Boolean-Arrays-and-Masks.ipynb).

下表列出NumPy實現的運算符號及對應的ufunc函數：除此之外還有布爾和二進制位操作；我們會在[比較，遮蓋和布爾邏輯](02.06-Boolean-Arrays-and-Masks.ipynb)中介紹它們。

| 運算符	    | 對應的ufunc函數    | 說明                           |
|---------------|---------------------|---------------------------------------|
|``+``          |``np.add``           |加法 (例如 ``1 + 1 = 2``)         |
|``-``          |``np.subtract``      |減法 (例如 ``3 - 2 = 1``)      |
|``-``          |``np.negative``      |一元取負 (例如 ``-2``)          |
|``*``          |``np.multiply``      |乘法 (例如 ``2 * 3 = 6``)   |
|``/``          |``np.divide``        |除法 (例如 ``3 / 2 = 1.5``)       |
|``//``         |``np.floor_divide``  |整除 (例如 ``3 // 2 = 1``)  |
|``**``         |``np.power``         |求冪 (例如 ``2 ** 3 = 8``)  |
|``%``          |``np.mod``           |模除 (例如 ``9 % 4 = 1``)|

### Absolute value 絕對值

> Just as NumPy understands Python's built-in arithmetic operators, it also understands Python's built-in absolute value function:

就像NumPy能夠理解Python內建的算術操作一樣，它同樣能理解Python內建的絕對值函數：

In [95]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

array([2, 1, 0, 1, 2])

In [96]:
np.absolute(x)  # NumPy's ufunc

array([2, 1, 0, 1, 2])

In [97]:
np.abs(x)  # NumPy的ufunc short

array([2, 1, 0, 1, 2])

In [98]:
#這個ufunc可以處理複數，返回的是矢量的長度：
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

array([5., 5., 2., 1.])

### Trigonometric functions 三角函數

> NumPy provides a large number of useful ufuncs, and some of the most useful for the data scientist are the trigonometric functions.
We'll start by defining an array of angles:

NumPy提供了大量的有用的ufuncs，對於數據科學加來說非常有用的還包括三角函數。我們先定義一個角度的數組：

In [99]:
theta = np.linspace(0, np.pi, 3)
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta)) # 正弦
print("cos(theta) = ", np.cos(theta)) # 餘弦
print("tan(theta) = ", np.tan(theta)) # 正切

theta      =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


In [100]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x)) # 反正弦
print("arccos(x) = ", np.arccos(x)) # 反餘弦
print("arctan(x) = ", np.arctan(x)) # 反正切

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### Exponents and logarithms 指數和對數

> Another common type of operation available in a NumPy ufunc are the exponentials:

NumPy中另一種常用操作是指數：

In [101]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [ 2.71828183  7.3890561  20.08553692]
2^x   = [2. 4. 8.]
3^x   = [ 3  9 27]


In [102]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [0.         0.69314718 1.38629436 2.30258509]
log2(x)  = [0.         1.         2.         3.32192809]
log10(x) = [0.         0.30103    0.60205999 1.        ]


In [103]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) = [0.         0.0009995  0.00995033 0.09531018]


### Specialized ufuncs 特殊的ufuncs

> NumPy has many more ufuncs available, including hyperbolic trig functions, bitwise arithmetic, comparison operators, conversions from radians to degrees, rounding and remainders, and much more.
A look through the NumPy documentation reveals a lot of interesting functionality.

NumPy包含更多的ufuncs，包括雙曲函數，二進制位運算，比較操作，角度弧度轉換，舍入以及求餘數等等。參考NumPy的在線文檔你可以看到很多有趣的函數說明。

> Another excellent source for more specialized and obscure ufuncs is the submodule ``scipy.special``.
If you want to compute some obscure mathematical function on your data, chances are it is implemented in ``scipy.special``.
There are far too many functions to list them all, but the following snippet shows a couple that might come up in a statistics context:

在`scipy.special`模塊中還有更多的特殊及難懂的ufuncs。如果你需要計算使用到晦澀數學函數操作你的數據，基本上你都可以在這個模塊中找到。下面列出了部分與數據統計相關的ufuncs，還有很多因為篇幅關係並未列出。

In [104]:
from scipy import special

In [105]:
# 伽瑪函數（通用階乘函數）及相關函數
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x)) # 伽瑪函數
print("ln|gamma(x)| =", special.gammaln(x)) # 伽瑪函數的自然對數
print("beta(x, 2)   =", special.beta(x, 2)) # 貝塔函數（第一類歐拉積分）

gamma(x)     = [1.0000e+00 2.4000e+01 3.6288e+05]
ln|gamma(x)| = [ 0.          3.17805383 12.80182748]
beta(x, 2)   = [0.5        0.03333333 0.00909091]


In [106]:
# 誤差函數 (高斯函數積分) 
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x)) # 誤差函數
print("erfc(x) =", special.erfc(x)) # 互補誤差函數
print("erfinv(x) =", special.erfinv(x)) # 逆誤差函數

erf(x)  = [0.         0.32862676 0.67780119 0.84270079]
erfc(x) = [1.         0.67137324 0.32219881 0.15729921]
erfinv(x) = [0.         0.27246271 0.73286908        inf]


## Advanced Ufunc Features

## 高級Ufunc特性

> Many NumPy users make use of ufuncs without ever learning their full set of features.
We'll outline a few specialized features of ufuncs here.

許多NumPy用戶在使用ufuncs的時候都沒有了解它們完整特性。我們在這裡會簡單介紹一些特別的特性。

### Specifying output 指定輸出

> For large calculations, it is sometimes useful to be able to specify the array where the result of the calculation will be stored.
Rather than creating a temporary array, this can be used to write computation results directly to the memory location where you'd like them to be.
For all ufuncs, this can be done using the ``out`` argument of the function:

對於大數據量的計算，有時指定存儲輸出數據的數組是很有用的。指定輸出結果的內存位置能夠避免創建臨時的數組。所有的ufuncs都能通過指定`out`參數來指定輸出的數組。

In [107]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y) # 指定結果存儲在y數組中
print(y)

[ 0. 10. 20. 30. 40.]


In [108]:
y = np.zeros(10)
np.power(2, x, out=y[::2]) # 指定结果存储在y数组中，每隔一个元素存一个
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


> If we had instead written ``y[::2] = 2 ** x``, this would have resulted in the creation of a temporary array to hold the results of ``2 ** x``, followed by a second operation copying those values into the ``y`` array.
This doesn't make much of a difference for such a small computation, but for very large arrays the memory savings from careful use of the ``out`` argument can be significant.

如果你沒使用`out`參數，而是寫成`y[::2] = 2 ** x`，這回導致首先創建一個臨時數組用來存儲`2 ** x`，然後再將這些值複製到y數組中。對於上面這麼小的數組來說，其實沒有什麼區別，但是如果對像是一個非常大的數組，使用`out`參數能節省很多內存空間。

### Aggregates 聚合

> For binary ufuncs, there are some interesting aggregates that can be computed directly from the object.
For example, if we'd like to *reduce* an array with a particular operation, we can use the ``reduce`` method of any ufunc. A reduce repeatedly applies a given operation to the elements of an array until only a single result remains.For example, calling ``reduce`` on the ``add`` ufunc returns the sum of all elements in the array:

對於二元運算ufuncs來說，還有一些很有趣的聚合函數可以直接從數組中計算出結果。例如，如果你想`reduce`一個數組，你可以對於任何ufuncs應用`reduce`方法。 reduce會重複在數組的每一個元素進行ufunc的操作，直到最後得到一個標量。例如，在`add` ufunc上調用`reduce`會返回所有元素的總和：

In [109]:
x = np.arange(1, 6)
np.add.reduce(x)

15

In [110]:
np.multiply.reduce(x)  #返回所有元素的乘積

120

In [111]:
np.add.accumulate(x)  #得到每一步計算得到的中間結果

array([ 1,  3,  6, 10, 15])

In [112]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120])

### Outer products 外積

> Finally, any ufunc can compute the output of all pairs of two different inputs using the ``outer`` method.
This allows you, in one line, to do things like create a multiplication table:

最後，任何ufunc都可以計算輸入的每一對元素的結果，使用`outer`方法。你可以一行代碼就完成類似創建乘法表的功能：

In [113]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

> The ``ufunc.at`` and ``ufunc.reduceat`` methods, which we'll explore in [Fancy Indexing](02.07-Fancy-Indexing.ipynb), are very helpful as well. Another extremely useful feature of ufuncs is the ability to operate between arrays of different sizes and shapes, a set of operations known as *broadcasting*.
This subject is important enough that we will devote a whole section to it (see [Computation on Arrays: Broadcasting](02.05-Computation-on-arrays-broadcasting.ipynb)).

Ufuncs還有一個極端有用的特性，能讓ufuncs在不同長度和形狀的數組之間進行計算，這是一組被稱為*廣播*的方法。這是一個非常重要的內容，因此我們會專門在[在數組上計算：廣播](02.05-Computation-on-arrays-broadcasting.ipynb)小節中進行介紹。`ufunc.at`和`ufunc.reduceat`方法也非常有用，我們會在[高級索引](02.07-Fancy-Indexing.ipynb)中詳細討論。